In [29]:
# First import necessary libraries
import pandas as pd
from sklearn.ensemble import StackingRegressor
import numpy as np

# Decision trees
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Neural networks
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Add, Input, Dense, Dropout, BatchNormalization, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from keras import layers
from keras.layers import concatenate
from tensorflow.keras import regularizers
from keras.regularizers import l1
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler

# Wrapper to make neural network compitable with StackingRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Linear model as meta-learn
from sklearn.linear_model import LinearRegression

# Create generic dataset for regression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

In [30]:
df= pd.read_csv("hourly_load_data_1.csv")
df.head()

,Total_occupant,Total_load
0,0,3.356876
1,0,3.502624
2,0,3.726831
3,0,2.931956
4,0,2.881140


In [31]:
df.shape

(2592, 2)

In [32]:
df.isnull().sum()

Total_occupant    0
Total_load        0
dtype: int64

In [33]:
df.dropna(inplace=True)

In [34]:
df.describe()

,Total_occupant,Total_load
count,2592.000000,2592.000000
mean,33.295139,7.412673
std,92.936729,17.869472
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,3.069164
max,833.000000,96.526306


In [35]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['Total_load']])

In [36]:
X_train1, X_test1, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42,shuffle=False)
print('Train set shape', X_train1.shape)
print('TEST set shape', X_test1.shape)

Train set shape (1814, 2)
TEST set shape (778, 2)


In [37]:
X_train_series=np.reshape(X_train1,(1814,2,1))
X_test_series=np.reshape(X_test1,(778,2,1))
print('Train set shape', X_train_series.shape)
print('Test set shape', X_test_series.shape)

Train set shape (1814, 2, 1)
Test set shape (778, 2, 1)


In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [39]:

def create_neural_network():
    
   model= Sequential()
   model.add(LSTM(units = 96, activation='relu', input_shape=(2,1),return_sequences=True))
   model.add(Dropout(0.1))
   model.add(LSTM(units = 96, activation='relu',return_sequences=True))
   model.add(Dropout(0.1))
   model.add(LSTM(units = 64, activation='relu',return_sequences=True))
   model.add(Dropout(0.1))
   model.add(LSTM(units = 32, activation='relu',return_sequences=True,))
   model.add(Dropout(0.1))
   model.add(LSTM(units = 32, activation='relu'))
   model.add(Dropout(0.1))
# Adding the output layer   
   model.add(Dense(units = 1,activation="linear"))     

   model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
    
    
   
   return model

In [40]:
def get_stacking(input_shape=None):

   
    level0 = list()
   
    
    level0.append(('xg', XGBRegressor(tree_method='approx',subsamples=0.7,n_estimators=300,max_depth=12,learning_rate=0.5,colsample_bytree=0.6)))
    level0.append(('rf', RandomForestRegressor(n_estimators=20,min_samples_split=5,min_samples_leaf=2,max_samples= 0.75,max_features=0.6,max_depth= 2,bootstrap= True)))
    
    
    for i in range(1):
    
        keras_reg = KerasRegressor(
                create_neural_network, 
               
                epochs=5,
                verbose=2)
        keras_reg._estimator_type = "regressor"
        
        level0.append(('nn_{num}'.format(num=i), keras_reg))
   
    level1 = LinearRegression()
    
  
    model = StackingRegressor(estimators=level0, final_estimator=level1, verbose=1)
    return model

In [41]:

model = get_stacking()
model.fit(X_train1, Y_train)


C:\Users\sunrise\AppData\Local\Temp\ipykernel_8568\2401107453.py:13: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = KerasRegressor(
c:\Users\sunrise\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_stacking.py:865: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[14:02:29] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "subsamples" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Epoch 1/5
57/57 - 12s - loss: 0.0359 - root_mean_squared_error: 0.1895 - 12s/epoch - 205ms/step
Epoch 2/5
57/57 - 1s - loss: 0.0342 - root_mean_squared_error: 0.1849 - 564ms/epoch - 10ms/step
Epoch 3/5
57/57 - 1s - loss: 0.0329 - root_mean_squared_error: 0.1815 - 572ms/epoch - 10ms/step
Epoch 4/5
57/57 - 1s - loss: 0.0322 - root_mean_squared_error: 0.1795 - 564ms/epoch - 10ms/step
Epoch 5/5
57/57 - 1s - loss: 0.0318 - root_mean_squared_error: 0.1784 - 580ms/epoch - 10ms/step
[14:02:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[14:02:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "subsamples" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:02:47] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "subsamples" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:02:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "subsamples" } might not be used.

  This coul

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/5
46/46 - 7s - loss: 0.0096 - root_mean_squared_error: 0.0980 - 7s/epoch - 155ms/step
Epoch 2/5
46/46 - 0s - loss: 0.0092 - root_mean_squared_error: 0.0961 - 440ms/epoch - 10ms/step
Epoch 3/5
46/46 - 0s - loss: 0.0091 - root_mean_squared_error: 0.0953 - 444ms/epoch - 10ms/step
Epoch 4/5
46/46 - 0s - loss: 0.0090 - root_mean_squared_error: 0.0950 - 448ms/epoch - 10ms/step
Epoch 5/5
46/46 - 0s - loss: 0.0090 - root_mean_squared_error: 0.0949 - 448ms/epoch - 10ms/step
12/12 - 1s - 852ms/epoch - 71ms/step
Epoch 1/5
46/46 - 7s - loss: 0.0358 - root_mean_squared_error: 0.1891 - 7s/epoch - 161ms/step
Epoch 2/5
46/46 - 0s - loss: 0.0347 - root_mean_squared_error: 0.1862 - 496ms/epoch - 11ms/step
Epoch 3/5
46/46 - 0s - loss: 0.0338 - root_mean_squared_error: 0.1839 - 460ms/epoch - 10ms/step
Epoch 4/5
46/46 - 0s - loss: 0.0333 - root_mean_squared_error: 0.1825 - 464ms/epoch - 10ms/step
Epoch 5/5
46/46 - 0s - loss: 0.0330 - root_mean_squared_error: 0.1817 - 456ms/epoch - 10ms/step
12/12 -

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   53.7s finished


StackingRegressor(estimators=[('xg',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.6,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.5, max_bin=None,
                                            m...
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None,
                                            reg_alpha=None, reg_lambda=None, ...)),
                              ('rf',
                               RandomForestRegressor(max_depth=2,
                                                     max_features=0.6,
                                                     max_samples=0.75,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=20)),
                              ('nn_0',
                               <keras.wrappers.scikit_learn.KerasRegressor object at 0x00000273F0F4EA10>)],
                  final_estimator=LinearRegression(), verbose=1)

In [42]:
prediction=model.predict(X_test1)

25/25 - 1s - 950ms/epoch - 38ms/step


In [43]:
prediction


array([-1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -1.36848877e-02, -1.36848877e-02, -1.36848877e-02,
       -1.36848877e-02, -

In [44]:
prediction.shape

(778,)

In [45]:
Prediction = prediction.reshape(prediction.shape[0], 1)

In [46]:
Prediction.shape

(778, 1)

In [47]:
Pred= s2.inverse_transform(Prediction)

In [48]:
Y_test

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [49]:
Y_test.shape

(778, 1)

In [50]:
Actual= s2.inverse_transform(Y_test)

In [59]:
df=pd.DataFrame(Pred,columns=["Prediciton"])
data=pd.DataFrame(Actual,columns=['Actual'])

In [60]:
df.to_csv(r"E:\Mam_iqra second paper\School of Design and Environment 4 (SDE4), Singapor\Stack_Model\prediction.csv")
data.to_csv(r"E:\Mam_iqra second paper\School of Design and Environment 4 (SDE4), Singapor\Stack_Model\Actual.csv")

In [1]:
import pandas as pd 
import numpy as np
df=pd.read_csv("output.csv")
features= list(df)
print("cloumn_names:",features)
z= df["Actual"]
x= df["Prediction"]
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE
max = x.max()
min = x.min()
NRMSE= (RMSE/(max-min))*100


print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)
print("Normalized Root Mean Square Error",NRMSE)

cloumn_names: ['Prediction', 'Actual']
Root Mean Square Error: 4.477010655306346
Mean Square Error: 20.043624407726558
Mean Absolute Error: 2.8530622389498714
Normalized Root Mean Square Error 6.209453512339133
